In [1]:
### FILE SIZE CHECKS ###
import os
def getFolderSize(folder):
    total_size = os.path.getsize(folder)
    for item in os.listdir(folder):
        itempath = os.path.join(folder, item)
        if os.path.isfile(itempath):
            total_size += os.path.getsize(itempath)
        elif os.path.isdir(itempath):
            total_size += getFolderSize(itempath)
    return total_size

dataAmountRAW = len(next(os.walk('HESEL_data'))[1])
dataAmountProcessed = len(next(os.walk('HESEL_processed'))[1])
dataSizeRAW = getFolderSize("HESEL_data")
dataSizeProcessed = getFolderSize("HESEL_processed")

print(f"Amount of RAW folders: {dataAmountRAW}")
print(f"Total file size: {dataSizeRAW/1e12} TB")
print()
print(f"Amount of processed folders: {dataAmountProcessed}")
print(f"Total file size: {dataSizeProcessed/1e9} GB")


Amount of RAW folders: 0
Total file size: 1e-12 TB

Amount of processed folders: 0
Total file size: 1e-09 GB


In [2]:
### FOLDER STATE CHECKS ###
### Libraries imports
import sys
import os
import numpy as np

### Locals imports
# Import paths file
pathPath = "os.environ['HOME']"
sys.path.append(pathPath)
import paths

# Import job submission script
sys.path.append(paths.scriptPath)

# Import scripts
from sweeps import nameGenerator

# Create the lists for data for the corresponding datasets
listCombies, nameParamList, nameDataList, listProcessed = [], [], [], []


# Add the data - these are the folders to check
#listCombies, nameParamList, nameDataList, listProcessed = nameGenerator({"INNERTEMP" : [670, 850, 20], "INNERDENSITY" : [3.0, 4.0, 0.1], "NOUT" : 1000}, customSuffix = f"GridSweep", roundParameters = {"INNERDENSITY" : 2})
##listCombies, nameParamList, nameDataList, listProcessed = nameGenerator({"INNERTEMP" : [670, 850, 20], "INNERDENSITY" : [4.1, 5.0, 0.1], "NOUT" : 1000}, customSuffix = f"GridSweep", roundParameters = {"INNERDENSITY" : 2})

# for i in range(5):#range(3):
#     listCombiesTEMP, nameParamListTEMP, nameDataListTEMP, listProcessedTEMP = nameGenerator({"INNERTEMP" : [100, 2000, 50], "NOUT" : [1000]}, customSuffix = f"P_loss_Curve{i}")
#     listCombies.append(listCombiesTEMP)
#     nameParamList.append(nameParamListTEMP)
#     nameDataList.append(nameDataListTEMP)
#     listProcessed.append(listProcessedTEMP)

for i in range(2):
    listCombiesTEMP, nameParamListTEMP, nameDataListTEMP, listProcessedTEMP = nameGenerator({"INNERTEMP" : [100, 1000, 50], "NOUT" : 1000}, customSuffix = f"Curve_TemplateCheck_{i}")
    listCombies.append(listCombiesTEMP)
    nameParamList.append(nameParamListTEMP)
    nameDataList.append(nameDataListTEMP)
    listProcessed.append(listProcessedTEMP)

# Convert to np arrays
listCombies = np.array(listCombies)
nameParamList = np.array(nameParamList)
nameDataList = np.array(nameDataList)
listProcessed = np.array(listProcessed)

# Now flatten them
listCombies = listCombies.flatten()
nameParamList = nameParamList.flatten()
nameDataList = nameDataList.flatten()
listProcessed = listProcessed.flatten()

# Now check for the first dump file in each folder
allGood = True
badList = []
print("===== Checking RAW data =====\n")
for i in range(len(nameDataList)):
    fPath = f"{paths.dataPath}/{nameDataList[i]}/BOUT.dmp.0.nc"
    if os.path.isfile(fPath) == False:
        allGood = False
        print(f"Missing data in: {nameDataList[i]}")
        bad = listCombies[i]
        for key in bad.keys():
            if isinstance(bad[key], np.floating):
                bad[key] = float(bad[key])
            if isinstance(bad[key], np.integer):
                bad[key] = int(bad[key])
        badList.append(bad)

if allGood == True:
    print("===== No missing RAW data was found =====\n\n")
else:
    print("\n===== End of list =====\n\n")
    print(f"Missing data parameter combinations: {badList}\n\n")

allGood = True
print("===== Checking processed data =====\n")
for i in range(len(nameDataList)):
    fPath = f"{paths.processedPath}/{nameDataList[i]}/Gamma_perp_averaged_z.npy"
    if os.path.isfile(fPath) == False:
        allGood = False
        print(f"Missing data in: {nameDataList[i]}")

if allGood == True:
    print("===== No missing processed data was found =====")
else:
    print("\n===== End of list=====")

===== Checking RAW data =====

===== No missing RAW data was found =====


===== Checking processed data =====

===== No missing processed data was found =====
